In [1]:
import matplotlib.pyplot as plt
import time 
import numpy as np
from astropy.io import fits
import os

In [2]:
directorio='/Users/oscar/Documents/Documentos/Docs/LME/CCD/OscuraImagenes/'
contenido = os.listdir(directorio)

In [ ]:
# Loop sobre todas las imagenes, se guarda en img_total cada ccd en fila demultiplexado
st = time.time()
#nsamp=112
#1200= tamx/nsamp
#img_individual= np.zeros((tamx, tamy))
for item in range(0,len(contenido)):
    hdulist = fits.open(directorio+contenido[item])
    tamx=int(hdulist[4].header['NAXIS1'])
    tamy=int(hdulist[4].header['NAXIS2'])
    nsamp=int(hdulist[4].header['NSAMP'])
    scidata = hdulist[4].data
    img_total= np.zeros((tamy, tamx))
    for i in range(0,nsamp):
        for k in range (0,int(tamx/nsamp)):
            img_total[:,k+i]=scidata[:,(k*nsamp)+i]
    print('Se termino la imagen ',item)   
    #plt.figure()
    #plt.imshow(img_total, cmap='gray')
    #plt.colorbar()
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

In [ ]:
#Prueba para imprimir una imagen 
#Faltan ajustes
for k in range (0,1200):
    img_individual[:,k]=scidata[:,(k*112)+110]
plt.figure()
plt.imshow(img_individual, cmap='gray')
plt.colorbar()

In [8]:
import concurrent.futures

In [9]:
#Probar paralelizando

#Funcion a paralelizar demux
def demux(nombre):
    hdulist = fits.open(nombre)
    tamx=int(hdulist[4].header['NAXIS1'])
    tamy=int(hdulist[4].header['NAXIS2'])
    nsamp=int(hdulist[4].header['NSAMP'])
    scidata = hdulist[4].data
    img_total= np.zeros((tamy, tamx))
    for i in range(0,nsamp):
        for k in range (0,int(tamx/nsamp)):
            img_total[:,k+i]=scidata[:,(k*nsamp)+i]
    print('Se termino la imagen ') 
    

In [10]:
##Main
if __name__ == '__main__':    
    directorio='/Users/oscar/Documents/Documentos/Docs/LME/CCD/OscuraImagenes/'
    contenido = os.listdir(directorio)
    for j in range(0,len(contenido)):
        contenido[j]=directorio+contenido[j]
    t1 = time.perf_counter()
    #with concurrent.futures.ProcessPoolExecutor() as executor:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(demux, contenido)
    t2 = time.perf_counter()
    print(f'Finished in {t2-t1} seconds')


Se termino la imagen 
Se termino la imagen 
Se termino la imagen 
Se termino la imagen 
Se termino la imagen 
Finished in 10.154880999999932 seconds


In [11]:
print(contenido)

['/Users/oscar/Documents/Documentos/Docs/LME/CCD/OscuraImagenes/ANSAMP1_clean_8.fz', '/Users/oscar/Documents/Documentos/Docs/LME/CCD/OscuraImagenes/ANSAMP1_clean_9_.fz', '/Users/oscar/Documents/Documentos/Docs/LME/CCD/OscuraImagenes/ANSAMP1_VoltsV2_clean_10.fz', '/Users/oscar/Documents/Documentos/Docs/LME/CCD/OscuraImagenes/ANSAMP400_CleanAndReadSeq_2.fz', '/Users/oscar/Documents/Documentos/Docs/LME/CCD/OscuraImagenes/ANSAMP400_VoltsV2_11.fz']
